In [225]:
# The code was removed by Watson Studio for sharing.

# Portfolio Allocation
Solver notebook

Requires:
* dse-do-utils
* decision-optimization-client

In [226]:
import pandas as pd

# Load data from scenario

In [227]:
#dd-ignore
from dse_do_utils import ScenarioManager
DO_MODEL_NAME = 'PortfolioAllocation'
SCENARIO_NAME = 'Scenario 1'

# from dse_do_utils import ScenarioManager
sm = ScenarioManager(model_name=DO_MODEL_NAME, scenario_name=SCENARIO_NAME, project=project)
inputs, outputs = sm.load_data_from_scenario()
sm.print_table_names()

Input tables: covariance, investment, parameters
Output tables: stats, kpis, investmentAllocation, portfolioKPIs


Get input data from inputs

In [229]:
all_investments = inputs['investment'].set_index(['id'], verify_integrity=True)
all_investments.head()

,expected_return,recommendation,country,industry,index,stock_price,expected_return_amount
id,,,,,,,
Invest0,1.00125,Strong Sell,FR,Utilities,0,161.150,161.351438
Invest1,1.56359,Buy,IT,Utilities,1,83.460,130.497221
Invest2,1.19330,Neutral,DE,Chemicals,2,24.380,29.092654
Invest3,1.80874,Strong Buy,DE,Chemicals,3,76.555,138.468091
Invest4,1.58501,Buy,FR,Insurance,4,122.300,193.846723


In [230]:
covariance = inputs['covariance'].set_index(['investment_1', 'investment_2'], verify_integrity=True)
covariance.head()

,,value
investment_1,investment_2,
Invest0,Invest0,10.950000
Invest1,Invest0,-0.119083
Invest2,Invest0,-0.008911
Invest3,Invest0,0.531663
Invest4,Invest0,0.601764


# Pre-processing

## Exclude 'Strong Sell'

In [231]:
investments = all_investments.query("recommendation != 'Strong Sell'").copy() # Copy so that we can add columns without a warning
print(f"Removed {len(all_investments)-len(investments)} Strong Sell investments")

Removed 2 Strong Sell investments


# Model

In [232]:
from docplex.mp.model import Model
mdl = Model(name='PortfolioAllocation')

# Decision Variables

Decision variables (per investment):
* $xInvest$ : float - the value of the investment
* $xInvestShares$ : integer - the number of shares
* $xInvestSelect$: boolean - True if invested


In [233]:
investments['xInvestShares'] = pd.Series(mdl.integer_var_list(investments.index, lb=0, name='xInvestShares'), index = investments.index)  # Integer number of shares
investments['xInvest'] = pd.Series(mdl.semicontinuous_var_list(investments.index, lb=10, ub=1000, name='xInvest'), index = investments.index)  # Integer number of shares
investments['xInvestSelect'] = pd.Series(mdl.binary_var_list(investments.index, name='xSelect'), index = investments.index)

# KPIs and Objective

The investment table column `expected_return` is the multiplied with the investment value to compute the expected return value of the investment.

$$
\begin{align}
expectedReturnAmount &= \sum\limits_{\forall i} xInvest_{i} * expectedReturn_i &&\qquad \text{with }  i \in investments\\
\end{align}
$$

In [234]:
expected_return_kpi = mdl.sum(investments.xInvest * investments.expected_return)
mdl.add_kpi(expected_return_kpi, "ExpectedReturn")

DecisionKPI(name=ExpectedReturn,expr=1.564xInvest_Invest1+1.193xInvest_Invest2+1.809xInvest_Invest3+1..)

$$
\begin{align}
margin &= \sum\limits_{\forall p, c} xDeliver_{p,c} * margin_p &&\qquad \text{with }  p \in product, c \in clients\\
\end{align}
$$

## Variance

variance = mdl.sum(float(df_var[s1][s2]) * fracs[s1] * fracs[s2] for s1 in df_stocks.index for s2 in df_stocks.index)<br>


$$
\begin{align}
variance &= \sum\limits_{\forall i,j} xInvest_{i} * xInvest_{j} * covariance_{i,j} &&\qquad \text{with }  i,j \in investments\\
\end{align}
$$

In [235]:
covariance_join = (covariance
                   .copy()
                   .reset_index()
                   .merge(investments[['xInvest']].reset_index(), left_on='investment_1', right_on='id')
                   .rename(columns={'xInvest':'xInvest_1'})
                   .drop(columns=['id'])
                   .merge(investments[['xInvest']].reset_index(), left_on='investment_2', right_on='id')
                   .rename(columns={'xInvest':'xInvest_2'})
                   .drop(columns=['id'])
                  )
covariance_join.head()

,investment_1,investment_2,value,xInvest_1,xInvest_2
0,Invest1,Invest1,9.817770,xInvest_Invest1,xInvest_Invest1
1,Invest2,Invest1,-0.677206,xInvest_Invest2,xInvest_Invest1
2,Invest3,Invest1,0.008789,xInvest_Invest3,xInvest_Invest1
3,Invest4,Invest1,-0.275887,xInvest_Invest4,xInvest_Invest1
4,Invest5,Invest1,0.587909,xInvest_Invest5,xInvest_Invest1


In [236]:
covariance_kpi = mdl.sum(covariance_join.value * covariance_join.xInvest_1 * covariance_join.xInvest_2)
_ = mdl.add_kpi(covariance_kpi, 'Covariance')

DecisionKPI(name=Covariance,expr=9.818xInvest_Invest1^2-1.354xInvest_Invest1*xInvest_Invest2+0.01..)

In [237]:
mdl.maximize(expected_return_kpi - 0.005 * covariance_kpi)

# Constraints

## Synchronization - Invest only whole number of shares

$$
\begin{align}
xInvestShares_{i} &=  xInvest_{i} * stockPrice_i &&\qquad \text{with }  i \in investments\\
\end{align}
$$

In [256]:
_ = mdl.add_constraints([investment.xInvest == investment.xInvestShares * investment.stock_price for investment in investments.itertuples()])

AttributeError: 'Pandas' object has no attribute 'xInvest'

## Select binary dvar
If investing in a investment, then it must be above a minimum and below a maximum.

$$
\begin{align}
xInvest_{i} &>=  xInvestSelect_{i} * investLb &&\qquad \text{with }  i \in investments\\
xInvest_{i} &<=  xInvestSelect_{i} * investUb &&\qquad \text{with }  i \in investments\\
\end{align}
$$

In [239]:
invest_lb = 10
invest_ub = 1000
_ = mdl.add_constraints([investment.xInvest >= investment.xInvestSelect * invest_lb
                     for investment in investments.itertuples()])
_ = mdl.add_constraints([investment.xInvest <= investment.xInvestSelect * invest_ub
                     for investment in investments.itertuples()])

[docplex.mp.LinearConstraint[](xInvest_Invest1,LE,1000xSelect_Invest1),
 docplex.mp.LinearConstraint[](xInvest_Invest2,LE,1000xSelect_Invest2),
 docplex.mp.LinearConstraint[](xInvest_Invest3,LE,1000xSelect_Invest3),
 docplex.mp.LinearConstraint[](xInvest_Invest4,LE,1000xSelect_Invest4),
 docplex.mp.LinearConstraint[](xInvest_Invest5,LE,1000xSelect_Invest5),
 docplex.mp.LinearConstraint[](xInvest_Invest6,LE,1000xSelect_Invest6),
 docplex.mp.LinearConstraint[](xInvest_Invest7,LE,1000xSelect_Invest7),
 docplex.mp.LinearConstraint[](xInvest_Invest8,LE,1000xSelect_Invest8),
 docplex.mp.LinearConstraint[](xInvest_Invest9,LE,1000xSelect_Invest9),
 docplex.mp.LinearConstraint[](xInvest_Invest10,LE,1000xSelect_Invest10),
 docplex.mp.LinearConstraint[](xInvest_Invest11,LE,1000xSelect_Invest11),
 docplex.mp.LinearConstraint[](xInvest_Invest12,LE,1000xSelect_Invest12),
 docplex.mp.LinearConstraint[](xInvest_Invest13,LE,1000xSelect_Invest13),
 docplex.mp.LinearConstraint[](xInvest_Invest14,LE,1000x

## Maximum budget

In [240]:
max_budget = 10000
mdl.add_constraint(mdl.sum(investments.xInvest) <= max_budget, 'MaxBudgetC')

docplex.mp.LinearConstraint[MaxBudgetC](xInvest_Invest1+xInvest_Invest2+xInvest_Invest3+xInvest_Invest4+xInvest_Invest5+xInvest_Invest6+xInvest_Invest7+xInvest_Invest8+xInvest_Invest9+xInvest_Invest10+xInvest_Invest11+xInvest_Invest12+xInvest_Invest13+xInvest_Invest14+xInvest_Invest16+xInvest_Invest17+xInvest_Invest18+xInvest_Invest19,LE,10000)

## Maximum and minimum investment per fund
If invest in fund, invest at least a minimum (10) and cap at maximum (1000)

## Max funds to invest

In [241]:
max_num_funds = 6
mdl.add_constraint(mdl.sum(investments.xInvestSelect) <= max_num_funds, 'MaxNumInvestmentsC')

docplex.mp.LinearConstraint[MaxNumInvestmentsC](xSelect_Invest1+xSelect_Invest2+xSelect_Invest3+xSelect_Invest4+xSelect_Invest5+xSelect_Invest6+xSelect_Invest7+xSelect_Invest8+xSelect_Invest9+xSelect_Invest10+xSelect_Invest11+xSelect_Invest12+xSelect_Invest13+xSelect_Invest14+xSelect_Invest16+xSelect_Invest17+xSelect_Invest18+xSelect_Invest19,LE,6)

## Investment mix constraints

### Max num investments per country

In [242]:
max_investments_per_country = 3
for country, group in investments.groupby('country'):
    mdl.add_constraint(mdl.sum(group.xInvestSelect) <= max_investments_per_country, 'MaxInvestmentsPerCountryC_{}'.format(country))

### Max num investments per industry

In [243]:
max_investments_per_industry = 2
for industry, group in investments.groupby('industry'):
    mdl.add_constraint(mdl.sum(group.xInvestSelect) <= max_investments_per_country, 'MaxInvestmentsPerIndustryC_{}'.format(industry))

# Solve

In [244]:
msol = mdl.solve(log_output=True, cplex_parameters = {'timelimit':30})
if msol is not None:
    mdl.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 2
CPXPARAM_TimeLimit                               30
Tried aggregator 1 time.
MIQP Presolve eliminated 13 rows and 0 columns.
MIQP Presolve modified 18 coefficients.
Reduced MIQP has 94 rows, 72 columns, and 230 nonzeros.
Reduced MIQP has 36 binaries, 18 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 324 nonzeros.
Presolve time = 0.01 sec. (0.16 ticks)
Probing time = 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 36 rows and 18 columns.
Reduced MIQP has 58 rows, 54 columns, and 158 nonzeros.
Reduced MIQP has 18 binaries, 18 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 324 nonzeros.
Presolve time = 0.00 sec. (0.19 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIQP has 58 rows, 54 columns, and 158 nonzeros.
Reduced MIQP has 18 

## Extract KPIs
Make sure to extract KPIs before doing the Pareto front. 
After the Pareto front loop, the KPIs will be those of the last run in the Pareto iteration.

In [245]:
all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
df_kpis = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
df_kpis

,kpi,value
0,ExpectedReturn,164.363744
1,Covariance,18118.268015


## Extract solution

In [246]:
investments['investment_allocation_decision'] = [dvar.solution_value for dvar in investments.xInvest]
investments['investment_pieces_decision'] = [dvar.solution_value for dvar in investments.xInvestShares]
investments['selected_investment_decision'] = [dvar.solution_value for dvar in investments.xInvestSelect]
investments = investments.drop(columns=['xInvestShares', 'xInvest', 'xInvestSelect'])

In [247]:
investments.head()

,expected_return,recommendation,country,industry,index,stock_price,expected_return_amount,investment_allocation_decision,investment_pieces_decision,selected_investment_decision
id,,,,,,,,,,
Invest1,1.56359,Buy,IT,Utilities,1,83.460,130.497221,0.000,0.0,0.0
Invest2,1.19330,Neutral,DE,Chemicals,2,24.380,29.092654,24.380,1.0,1.0
Invest3,1.80874,Strong Buy,DE,Chemicals,3,76.555,138.468091,0.000,0.0,0.0
Invest4,1.58501,Buy,FR,Insurance,4,122.300,193.846723,0.000,0.0,0.0
Invest5,1.47987,Buy,NL,Technology,5,7.646,11.315086,15.292,2.0,1.0


# Pareto front

Find a set of samples where:
* Given an expected return (x-axis)
* Find the minimum variance (y-axis)

Approach:
* Find range of return:
  * Maximize return (without covariance)
  * Minimize covariance, then maximize return (lex)
* Sample the return range
* Start with the return lower-range, run an optimization:
  * add/change constraint xReturn >= return_sample
  * minimize covariance
* Plot the Pareto curve

### Find return upper-range
Maximize(only) the return. Ignore covariance.

In [248]:
mdl.maximize(expected_return_kpi)
mdl.solve()
return_max_range = expected_return_kpi.solution_value
return_max_range

10608.3695527

### Find return lower-range
First, minimize the covariance KPI.<br>
Then fix this value and maximize the return.<br>
This will generate the lower-bound for the return.

In [249]:
mdl.minimize(covariance_kpi)
mdl.solve()
covariance_min_range = covariance_kpi.solution_value
# return_min_range_v1 = expected_return_kpi.solution_value
covariance_min_range

0.0

In [250]:
c = mdl.add_constraint(covariance_kpi <= covariance_min_range)
mdl.maximize(expected_return_kpi)
mdl.solve()
return_min_range = expected_return_kpi.solution_value
mdl.remove(c)
return_min_range

0.0

### Create a samples for Pareto front iteration
These are the (fixed) values for the return for which we'll minimize the covariance.

In [251]:
import numpy
# numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)
return_samples = numpy.linspace(start=return_min_range, stop=return_max_range, num=10, endpoint=True, retstep=False, dtype=None)
return_samples

array([    0.        ,  1178.70772808,  2357.41545616,  3536.12318423,
        4714.83091231,  5893.53864039,  7072.24636847,  8250.95409654,
        9429.66182462, 10608.3695527 ])

### Pareto front computation
* loop from high to low return
* Initially add one constraint: return_kpi > 0
* In each iteration, relax the rhs value

As a result:
* This ensures a warm start from a feasible value (and might be faster).
* We can easily re-run this loop.

In [252]:
data = []
mdl.minimize(covariance_kpi)
c = mdl.add_constraint(expected_return_kpi >= 0)  # Initialize constraint without restricting the value
for return_value in reversed(return_samples):
    print(f"return  = {return_value}")
    c.rhs = return_value  # The right-hand-side contains the constant
    mdl.solve()
    data.append({'return_sol':expected_return_kpi.solution_value, 'covariance_sol':covariance_kpi.solution_value})
# c.rhs = 0  # Or delete the constraint
mdl.remove(c)
pareto_df = pd.DataFrame(data)
pareto_df

return  = 10608.3695527
return  = 9429.661824622222
return  = 8250.954096544445
return  = 7072.246368466667
return  = 5893.538640388889
return  = 4714.830912311111
return  = 3536.1231842333336
return  = 2357.4154561555556
return  = 1178.7077280777778
return  = 0.0


,return_sol,covariance_sol
0,10608.369553,5.693218e+07
1,9430.675244,4.285024e+07
2,8252.416813,3.270995e+07
3,7072.566480,2.398704e+07
4,5897.826114,1.673948e+07
5,4723.009590,1.070670e+07
6,3542.929101,6.061079e+06
7,2361.504795,2.676674e+06
8,1180.475034,7.049170e+05
9,0.000000,0.000000e+00


In [253]:
#dd-ignore
import plotly.express as px

fig = px.line(df, y="return_sol", x="covariance_sol", title='Expected Return by Variance')
fig.show()

# Post processing

## Investment Allocation Report
investment_allocation_report:
* investment_id
* value
* investment_industry
* investment_country
* investment_recommendation
* investment_industry

In [254]:
outputs['InvestmentAllocation'] = investments.reset_index()
outputs['ParetoFront'] = pareto_df
outputs['PortfolioKPIs'] = df_kpis

In [255]:
#dd-ignore
sm.update_solve_output_into_scenario(mdl, outputs)